# Declare variables

In [1]:
from osgeo import gdal
import matplotlib.pyplot as plt
import numpy as np
import read_cover_fraction as rcf

In [2]:
def get_dataSG(fname,latmax,latmin,lonmax,lonmin):

    gdal.UseExceptions()
    ds = gdal.Open(fname)
    data = ds.ReadAsArray()
    gt = ds.GetGeoTransform()
    #
    #
    #
    xres = gt[1]
    yres = gt[5]
    #
    xmin = gt[0]
    ymin = gt[3]
    #
    xmax = gt[0] + (xres * ds.RasterXSize)
    ymax = gt[3] + (yres * ds.RasterYSize)
#    else:
#        xmax = gt[0] + (xres * ds.RasterXSize) 
#        ymax = gt[3] + (yres * ds.RasterYSize) 


    X=np.arange(xmin+xres,xmax+xres,xres)
    Y=np.arange(ymin+yres,ymax+yres,yres)


    #Chunck the data to save RAM for plotting
    #Ydecreasing
    b=np.min(np.where(Y<latmin)[0])
    a=np.max(np.where(Y>latmax)[0])

    c=np.max(np.where(X<lonmin)[0])
    d=np.min(np.where(X>lonmax)[0])

#    print(np.max(data))
#    print(np.min(data))
    #ECOCLIMAP data have no projections so no need to reproject the data
    #to be verified for other datasets
    LAT=Y[a:b]
    LON=X[c:d]
    D=data[a:b,c:d]

    return (LAT,LON,D)

In [3]:
#search the uniques cover values and creates a new coverlist using consecutive numbers
#    DSG: the dataset
#    covernum: cover number
#    covername : name of the cover
#    threshold : threshold to remove the cover with fewer points than the threshold (helps for clearer plot)
def find_uniques_new_values_cover(DSG,covernum,covername,threshold):
    uSG=np.unique(DSG)
    DSG2=np.zeros(DSG.shape)-1
    repeat=np.array([len(np.where(DSG==u)[0]) for u in uSG])
    muSG=uSG[np.where(repeat>threshold)[0]]
    uniqnames=[]
    for i,mu in enumerate(muSG):
        DSG2[np.where(DSG==mu)]=i*2
        if u==0:
            uniqnames.append('no data')
        else:
            indname=np.where(mu==covernum)[0]
            uniqnames.append(covername[indname])
            
    D=np.ma.masked_where(DSG2==-1,DSG2)
    return(D,muSG,uniqnames)
# creates a colormap regarding the length of the unique cover values uSG
# colorname: name of the desired colormap
def set_colormap(uSG,colorname):
    num_colors = len(uSG) 
    cm = plt.get_cmap(colorname) 
    scheme = [cm(i*1./num_colors) for i in range(num_colors)] 
#    bins = np.linspace(1, num_colors, num_colors)
    col = ListedColormap(scheme)
    return(col)
    

    
def plot_data(LON,LAT,D2,col,uniqnames,figurename):
    fig = plt.figure(figsize=(25, 25))
    ax=fig.gca()
    cp=plt.get_cmap(col,np.max(D2)-np.min(D2)+1)
    c=ax.pcolormesh(LON,LAT,D2,cmap=cp, vmin=np.min(D2)-1,vmax=np.max(D2)+1)
    cb=plt.colorbar(c, ticks=np.arange(np.min(D2),np.max(D2)+1),extendfrac='auto', spacing='proportional')
#    cb.set_ticks([i for i in range(0, len(uniqnames))])  
    cb.set_ticks([i for i in np.arange(0, 2*len(uniqnames)+1,2)]) 
    cb.set_ticklabels(uniqnames)
    cb.ax.tick_params(labelsize=20)
    ax.set_xlim(np.min(LON),np.max(LON))
    ax.set_ylim(np.min(LAT),np.max(LAT))
    ax.set_xlabel('Longitude',fontsize=20)
    ax.set_ylabel('Latitude',fontsize=20)
    ax.xaxis.set_tick_params(labelsize=20)
    ax.yaxis.set_tick_params(labelsize=20)
    fig.savefig(figurename,bbox_inches='tight',transparent=True)    
    return
    
#fname the name of the dir file check if the header is ok 
#https://hirlam.org/trac/wiki/PGD


latmax=55.5
latmin=50.9
lonmax=-5.4
lonmin=-11

fnameSG='/mnt/g/ECOCLIMAP/ECOCLIMAPSG/test_ecosg.tif'
(LATSG,LONSG,DSG)=get_dataSG(fnameSG,latmax,latmin,lonmax,lonmin)
covernamef='/mnt/g/ECOCLIMAP/ECOCLIMAPSG/ECOCLIMAP_SG_cover_data.csv'
#get the cover deatils from the csvfile
(fieldnames,covernum,covername,cityfrac,vegfrac,inwaterfrac,seafrac)=rcf.cover_fraction(covernamef)

In [4]:
fnstation='/mnt/g/Stations_list.txt'
data=open(fnstation,'r')

In [5]:
lat=[]
lon=[]
for d in data:
    lat.append(d.split(' ')[0].strip(' '))
    lon.append(d.split(' ')[1].strip(' '))
LAT=np.array(lat).astype(float)
LON=np.array(lon).astype(float)

In [6]:
Covname=[]

In [7]:
for x,_ in enumerate(LAT):
    idx=np.argmin(np.abs(LAT[x]-LATSG))
    idy=np.argmin(np.abs(LON[x]-LONSG))
    Covname.append(covername[int(DSG[idx,idy])-1])

In [8]:
fneco='/mnt/g/ECOCLIMAP/ECOCLIMAPSG_Stations_list.txt'
dataeco=open(fneco,'w')
dataeco.write('LAT, LON, Cover\n')
for x,_ in enumerate(LAT):
    dataeco.write(str(LAT[x])+', '+str(LON[x])+', '+Covname[x]+'\n')
dataeco.close()

In [9]:
fnameII = '/mnt/g/ECOCLIMAP/open_ECOCLIMAP2.3/ECOCLIMAP_II_EUROP_V2.3.dir'
(LATII,LONII,DII)=get_dataSG(fnameII,latmax,latmin,lonmax,lonmin)
filenamecoverII='/mnt/g/ECOCLIMAP/ECOCLIMAP_II_cover_data.csv'
(fieldnamesII,covernumII,covernameII,cityfracII,vegfracII,inwaterfracII,seafracII)=rcf.cover_fraction(filenamecoverII)

In [10]:
CovnameII=[]
for x,_ in enumerate(LAT):
    idx=np.argmin(np.abs(LAT[x]-LATII))
    idy=np.argmin(np.abs(LON[x]-LONII))
    CovnameII.append(covernameII[int(DII[idx,idy])-1])

In [11]:
fnII='/mnt/g/ECOCLIMAP/ECOCLIMAPII_Stations_list.txt'
dataII=open(fnII,'w')
dataII.write('LAT, LON, Cover\n')
for x,_ in enumerate(LAT):
    dataII.write(str(LAT[x])+', '+str(LON[x])+', '+CovnameII[x]+'\n')
dataII.close()

In [12]:
covernameII[0]

'Sea and ocean'

In [13]:


(DSG2,uSG,uniqnames)=find_uniques_new_values_cover(DSG,covernum,covername,1)

NameError: name 'u' is not defined

In [ ]:
## MAnually created colorbar for ECOSG

In [ ]:
a0=(0/255.0,0/255.0,0/255.0,255/255.0)## 0 value
a=(0/255.0,0/255.0,128/255.0,255/255.0)##navy blue 1.sea    
b=(0/255.0,0/255.0,205/255.0,255/255.0)## mediumblue 2.inland waters
c=(0/255.0, 0/255.0, 255/255.0, 255/255.0) ##Blue 3.rivers
d=(211/255.0,211/255.0,211/255.0,255/255.0)## lightgray 4.Bare land
e=(169/255.0,169/255.0,169/255.0,255/255.0)## darkgray 5.Rocks
f=(255/255.0,250/255.0,250/255.0,255/255.0) ## Snow 6.permanent snow
g=(240/255.0,255/255.0,240/255.0,255/255.0)## Honeydew 7.boreal broadleaf deciduous
h=(85/255.0,107/255.0,47/255.0,255/255.0)## darkolivegreen 8.temperate broadleaf deciduous
ii=(154/255.0,205/255.0,50/255.0,255/255.0)## yellowgreen 9.tropical broadleaf deciduous
k=(0/255.0,128/255.0,0/255.0,255/255.0)## green 10.temperate broadleaf evergreen
l=(255/255.0,127/255.0,80/255.0,255/255.0)## coral 11. tropical broadleaf evergreen
m=(160/255.0,82/255.0,45/255.0,255/255.0)## siena 12. boreal needleaf evergreen
n=(34/255.0,139/255.0,34/255.0,255/255.0)## forest green 13.temperate needleleaf evergreen
o= (188/255.0,143/255.0,143/255.0,255/255.0)## rosybrown 14. boreal needleleaf deciduous
p=(205/255.0,133/255.0,63/255.0,255/255.0)## peru 15. shrubs
q=(222/255.0,184/255.0,135/255.0,255/255.0)##  burlywood 16. boreal grassland
r=(50/255.0,205/255.0,50/255.0,255/255.0)##limegreen 17 . temperate grassland
s=(255/255.0,215/255.0,0/255.0,255/255.0) ##gold 18. tropical grassland
t=(32/255.0,178/255.0,170/255.0,255/255.0)##lightseagreen 19.winter crop
u=(173/255.0,255/255.0,47/255.0,255/255.0)##green yellow 20.summer crop
v=(189/255.0,183/255.0,107/255.0,255/255.0)##darkkhaki 21. C4 crops
w=(102/255.0,102/255.0,0/255.0,255/255.0)## Dark yellow3 22.flooded trees
x=(46/255.0,139/255.0,87/255.0,255/255.0)## seagreen 23.flooded grassland
y=(255/255.0,0/255.0,0/255.0,255/255.0)## 24. red LCZ1
z=(255/255.0,30/255.0,0/255.0,255/255.0)## 25. red LCZ2
a1=(255/255.0,60/255.0,0/255.0,255/255.0)## 26. red LCZ3
b1=(255/255.0,90/255.0,0/255.0,255/255.0)## 27. red LCZ4
c1=(255/255.0,120/255.0,0/255.0,255/255.0)## 28. red LCZ5
d1=(255/255.0,150/255.0,0/255.0,255/255.0)## 29. red LCZ6
e1=(255/255.0,180/255.0,0/255.0,255/255.0)## 30. red LCZ7
f1=(255/255.0,210/255.0,0/255.0,255/255.0)## 31.red LCZ8
g1=(255/255.0,240/255.0,0/255.0,255/255.0)## 32. red LCZ9
h1=(128/255.0,128/255.0,128/255.0,255/255.0)## 33. gray LCZ10




In [ ]:
scheme=[a0,a,b,c,d,e,f,g,h,ii,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,a1,b1,c1,d1,e1,f1,g1,h1]
schemevalueSG=[scheme[cov] for cov in uSG]
colSG = ListedColormap(schemevalueSG)
plot_data(LONSG,LATSG,DSG2,colSG,uniqnames,'ECOSG_Cover.png')
#

(DII2,uII,uniqnamesII)=find_uniques_new_values_cover(DII,covernumII,covernameII,100)
colII=set_colormap(uII,'jet')
plot_data(LONII,LATII,DII2,colII,uniqnamesII,'ECOII_Cover.png')
